In [65]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
from sqlalchemy.engine import url
import json
from sqlalchemy import extract
from sqlalchemy.engine import make_url
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
import psycopg2

In [66]:
with open('config_copy.json') as datafile:
    data = json.load(datafile)

env = data['dev']

In [67]:
db = env['db']
user = env['user']
password = env['pass']
port = env['port']
host = env['host']

In [68]:
engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [69]:
Base = automap_base()

In [70]:
Base.prepare(autoload_with = engine)

In [71]:
Base.classes.keys()

['jhu_cre_cases_deaths', 'location']

In [72]:
location = Base.classes.location

In [73]:
jhu_data = Base.classes.jhu_cre_cases_deaths

In [74]:
Session = sessionmaker(bind = engine)

session = Session()

In [75]:
sql = session.query(jhu_data)

In [76]:
jhucredf = pd.DataFrame(engine.connect().execute(text(str(sql))))

In [77]:
jhucredf

,jhu_cre_cases_deaths_fips,jhu_cre_cases_deaths_cases,jhu_cre_cases_deaths_deaths,jhu_cre_cases_deaths_lat,jhu_cre_cases_deaths_long,jhu_cre_cases_deaths_state,jhu_cre_cases_deaths_county,jhu_cre_cases_deaths_popuni,jhu_cre_cases_deaths_total_population,jhu_cre_cases_deaths_zero_rf,...,jhu_cre_cases_deaths_single_fathers_pop,jhu_cre_cases_deaths_plus_family_homes,jhu_cre_cases_deaths_highschool_grad,jhu_cre_cases_deaths_multilingual_5yrs_plus,jhu_cre_cases_deaths_full_time_workers,jhu_cre_cases_deaths_no_health_insur,jhu_cre_cases_deaths_internet_homes,jhu_cre_cases_deaths_no_vehicle,jhu_cre_cases_deaths_homeowner_vacancy,jhu_cre_cases_deaths_rental_vacancy
0,1001,19732,230,32.539527,-86.644082,Alabama,Autauga County,55688,55380,20573,...,723,111,49283,779,29291,3953,44884,2450,779,1726
1,1003,69641,724,30.727750,-87.722071,Alabama,Baldwin County,221898,212830,78622,...,2218,887,201483,3994,112058,19748,181512,7322,7322,96747
2,1005,7451,103,31.868263,-85.387129,Alabama,Barbour County,22023,25361,5024,...,220,132,16120,572,8170,2488,13323,2246,836,1629
3,1007,8067,109,32.996421,-87.125115,Alabama,Bibb County,20393,22493,6280,...,346,163,16130,265,8096,2182,14111,1264,305,1142
4,1009,18616,261,33.982109,-86.567906,Alabama,Blount County,57697,57681,18189,...,1038,115,46446,1961,27463,6231,42118,2365,403,4846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,12484,139,41.659439,-108.882788,Wyoming,Sweetwater County,41888,43521,16977,...,502,209,38872,1298,21739,4733,35185,712,879,12817
3138,56039,12123,16,43.935225,-110.589080,Wyoming,Teton County,23390,23280,7250,...,140,771,22267,1777,13028,2970,20372,561,467,5730
3139,56041,6378,43,41.287818,-110.547578,Wyoming,Uinta County,20183,20479,7744,...,322,141,18709,322,10091,2260,18063,686,100,2401
3140,56043,2749,50,43.904516,-107.680187,Wyoming,Washakie County,7738,8027,2601,...,108,77,6948,108,3869,1160,6051,379,15,1903


In [78]:
y = jhucredf["jhu_cre_cases_deaths_deaths"]
X = jhucredf.drop(columns=["jhu_cre_cases_deaths_deaths", "jhu_cre_cases_deaths_state", "jhu_cre_cases_deaths_county"])

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=20)

In [80]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [81]:
classifier.fit(X_train, y_train)

C:\Users\14405\New\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200, random_state=1)

In [82]:
y_pred = classifier.predict(X_test)

In [83]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.01272264631043257
